## 文本分类实例

### Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,Trainer,TrainingArguments
from datasets import load_dataset

### Step2 加载数据

In [2]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/7766 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

### Step3 划分数据集

In [3]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

### Step4 数据集预处理

In [4]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

### Step5 创建模型

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Step6 创建评估函数

In [15]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

### Step7 创建TrainingArguments

In [9]:
train_args = TrainingArguments(output_dir="./checkpoints")    
train_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=no,
evaluation_strategy=None,
fp16=False,
fp16_backend=auto,
fp16

In [16]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

d:\Anaconda3\envs\llm\lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
f

### Step8 创建Trainer

In [17]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

### Step9 模型训练

In [18]:
trainer.train()

  0%|          | 0/330 [00:00<?, ?it/s]

{'loss': 0.5948, 'grad_norm': 5.265766620635986, 'learning_rate': 1.9393939393939395e-05, 'epoch': 0.09}
{'loss': 0.5454, 'grad_norm': 2.48745059967041, 'learning_rate': 1.8787878787878792e-05, 'epoch': 0.18}
{'loss': 0.4582, 'grad_norm': 2.3169820308685303, 'learning_rate': 1.8181818181818182e-05, 'epoch': 0.27}
{'loss': 0.3777, 'grad_norm': 6.27378511428833, 'learning_rate': 1.7575757575757576e-05, 'epoch': 0.36}
{'loss': 0.373, 'grad_norm': 2.786111831665039, 'learning_rate': 1.6969696969696972e-05, 'epoch': 0.45}
{'loss': 0.3482, 'grad_norm': 4.767660617828369, 'learning_rate': 1.6363636363636366e-05, 'epoch': 0.55}
{'loss': 0.3263, 'grad_norm': 3.774388313293457, 'learning_rate': 1.575757575757576e-05, 'epoch': 0.64}
{'loss': 0.3757, 'grad_norm': 2.7259044647216797, 'learning_rate': 1.5151515151515153e-05, 'epoch': 0.73}
{'loss': 0.2953, 'grad_norm': 3.9339394569396973, 'learning_rate': 1.4545454545454546e-05, 'epoch': 0.82}
{'loss': 0.3247, 'grad_norm': 4.1619157791137695, 'learn

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.30656030774116516, 'eval_accuracy': 0.8635778635778636, 'eval_f1': 0.8962818003913894, 'eval_runtime': 0.9408, 'eval_samples_per_second': 825.879, 'eval_steps_per_second': 7.44, 'epoch': 1.0}
{'loss': 0.2918, 'grad_norm': 5.531589508056641, 'learning_rate': 1.2727272727272728e-05, 'epoch': 1.09}
{'loss': 0.288, 'grad_norm': 2.9826276302337646, 'learning_rate': 1.2121212121212122e-05, 'epoch': 1.18}
{'loss': 0.2408, 'grad_norm': 2.924783706665039, 'learning_rate': 1.1515151515151517e-05, 'epoch': 1.27}
{'loss': 0.2408, 'grad_norm': 6.96920919418335, 'learning_rate': 1.0909090909090909e-05, 'epoch': 1.36}
{'loss': 0.2971, 'grad_norm': 5.145956039428711, 'learning_rate': 1.0303030303030304e-05, 'epoch': 1.45}
{'loss': 0.2671, 'grad_norm': 3.031907558441162, 'learning_rate': 9.696969696969698e-06, 'epoch': 1.55}
{'loss': 0.2584, 'grad_norm': 2.9205284118652344, 'learning_rate': 9.090909090909091e-06, 'epoch': 1.64}
{'loss': 0.2417, 'grad_norm': 2.904542922973633, 'learning_

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.28934839367866516, 'eval_accuracy': 0.879021879021879, 'eval_f1': 0.907843137254902, 'eval_runtime': 0.9553, 'eval_samples_per_second': 813.363, 'eval_steps_per_second': 7.328, 'epoch': 2.0}
{'loss': 0.2209, 'grad_norm': 3.3963513374328613, 'learning_rate': 6.060606060606061e-06, 'epoch': 2.09}
{'loss': 0.1944, 'grad_norm': 4.925166130065918, 'learning_rate': 5.4545454545454545e-06, 'epoch': 2.18}
{'loss': 0.2038, 'grad_norm': 1.826399326324463, 'learning_rate': 4.848484848484849e-06, 'epoch': 2.27}
{'loss': 0.2158, 'grad_norm': 3.3820927143096924, 'learning_rate': 4.242424242424243e-06, 'epoch': 2.36}
{'loss': 0.2328, 'grad_norm': 4.075730323791504, 'learning_rate': 3.6363636363636366e-06, 'epoch': 2.45}
{'loss': 0.214, 'grad_norm': 3.6337063312530518, 'learning_rate': 3.0303030303030305e-06, 'epoch': 2.55}
{'loss': 0.2563, 'grad_norm': 2.9752421379089355, 'learning_rate': 2.4242424242424244e-06, 'epoch': 2.64}
{'loss': 0.2309, 'grad_norm': 5.160232067108154, 'learning

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.2873563766479492, 'eval_accuracy': 0.8738738738738738, 'eval_f1': 0.9052224371373307, 'eval_runtime': 0.9547, 'eval_samples_per_second': 813.846, 'eval_steps_per_second': 7.332, 'epoch': 3.0}
{'train_runtime': 73.3784, 'train_samples_per_second': 285.697, 'train_steps_per_second': 4.497, 'train_loss': 0.2917381889892347, 'epoch': 3.0}


TrainOutput(global_step=330, training_loss=0.2917381889892347, metrics={'train_runtime': 73.3784, 'train_samples_per_second': 285.697, 'train_steps_per_second': 4.497, 'total_flos': 351909933963264.0, 'train_loss': 0.2917381889892347, 'epoch': 3.0})

### Step10 模型评估

In [19]:
trainer.evaluate(tokenized_datasets["test"])

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.28934839367866516,
 'eval_accuracy': 0.879021879021879,
 'eval_f1': 0.907843137254902,
 'eval_runtime': 0.9377,
 'eval_samples_per_second': 828.599,
 'eval_steps_per_second': 7.465,
 'epoch': 3.0}

### Step11 模型预测

In [20]:
trainer.predict(tokenized_datasets["test"])

  0%|          | 0/7 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-1.3429664 ,  1.719666  ],
       [ 0.12806785, -0.08675361],
       [-0.74521714,  1.2515721 ],
       ...,
       [-1.7241703 ,  2.165235  ],
       [-2.283361  ,  2.6798043 ],
       [-1.7675138 ,  2.4500809 ]], dtype=float32), label_ids=array([1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1

In [21]:
from transformers import pipeline

id2_label = id2_label = {0: "差评！", 1: "好评！"}
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [22]:
sen = "我觉得不错！"
pipe(sen)

[{'label': '好评！', 'score': 0.9835914373397827}]